In [1]:
%pip install datasets

  Using cached pandas-2.3.3-cp312-cp312-macosx_11_0_arm64.whl.metadata (91 kB)
  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached httpcore-1.0.9-py3-none-any.whl.metadata (21 kB)
  Using cached h11-0.16.0-py3-none-any.whl.metadata (8.3 kB)
  Using cached aiohappyeyeballs-2.6.1-py3-none-any.whl.metadata (5.9 kB)
  Using cached aiosignal-1.4.0-py3-none-any.whl.metadata (3.7 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached httpx-0.28.1-py3-none-any.whl (73 kB)
Using cached httpcore-1.0.9-py3-none-any.whl (78 kB)
Using cached aiohappyeyeballs-2.6.1-py3-none-any.whl (15 kB)
Using cached aiosignal-1.4.0-py3-none-any.whl (7.5 kB)
Using cached h11-0.16.0-py3-none-any.whl (37 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.2/34.2 MB 31.6 MB/s  0:00:01 eta 0:00:01
Using cached sniffio-1.3.1-py3-none-any.

In [32]:
from datasets import load_dataset
df = load_dataset("microsoft/ms_marco", "v1.1")

In [ ]:
df['train']['passages'][0]

{'is_selected': [0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
 'passage_text': ["Since 2007, the RBA's outstanding reputation has been affected by the 'Securency' or NPA scandal. These RBA subsidiaries were involved in bribing overseas officials so that Australia might win lucrative note-printing contracts. The assets of the bank include the gold and foreign exchange reserves of Australia, which is estimated to have a net worth of A$101 billion. Nearly 94% of the RBA's employees work at its headquarters in Sydney, New South Wales and at the Business Resumption Site.",
  "The Reserve Bank of Australia (RBA) came into being on 14 January 1960 as Australia 's central bank and banknote issuing authority, when the Reserve Bank Act 1959 removed the central banking functions from the Commonwealth Bank. The assets of the bank include the gold and foreign exchange reserves of Australia, which is estimated to have a net worth of A$101 billion. Nearly 94% of the RBA's employees work at its headquarters in Sydn

In [41]:
%env PYTHONPATH=/Users/uvuv/Informational_Search
if os.environ["PYTHONPATH"] not in sys.path:
    sys.path.insert(0, os.environ["PYTHONPATH"])

import torch
from transformers import AutoModelForMaskedLM, AutoTokenizer
from spladev2.cloned.splade.splade.models.transformer_rep import Splade

env: PYTHONPATH=/Users/uvuv/Informational_Search


In [42]:
model_type_or_dir = "naver/splade-cocondenser-ensembledistil"

model = Splade(model_type_or_dir, agg="max")
model.eval()
tokenizer = AutoTokenizer.from_pretrained(model_type_or_dir)
reverse_voc = {v: k for k, v in tokenizer.vocab.items()}

In [ ]:
from typing import List
from tqdm import tqdm

mrr10 = 0
for index, passage in tqdm(enumerate(df['train']['passages'][:100])):
    passge_text: List[str] = passage['passage_text']
    query: str = df['train']['query'][index]
    with torch.no_grad():
        query_rep = model(d_kwargs=tokenizer(query, return_tensors="pt"))["d_rep"].squeeze()
        doc_rep = torch.empty(0)
        for text in passge_text:
            doc_rep = torch.cat([doc_rep, model(d_kwargs=tokenizer(text, return_tensors="pt"))["d_rep"]])

        relevant_indicies = torch.argsort(query_rep @ doc_rep.T, descending=True)
        result_index = torch.argmax(torch.tensor(df['train']['passages'][index]['is_selected']))
        rank = torch.where(relevant_indicies == result_index)
        mrr10 += 1 / (rank[0] + 1)
print(mrr10 / 100)
    

10it [00:08,  1.15it/s]

tensor([0.6917])


In [72]:
%pip install ir_datasets

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 866.1/866.1 kB 9.0 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 14.3 MB/s  0:00:00 eta 0:00:01
  Created wheel for cbor: filename=cbor-1.0.0-cp312-cp312-macosx_14_0_arm64.whl size=20300 sha256=e778be9d3f718c0f09877a2d2eb7c34d36eb1b879d0e9405865c13f01d344399
  Stored in directory: /Users/uvuv/Library/Caches/pip/wheels/44/3e/21/a739cbcc331a1ab45c326d6edbdac6118de4402f6076e30ff1
  Created wheel for warc3-wet-clueweb09: filename=warc3_wet_clueweb09-0.2.5-py3-none-any.whl size=18997 sha256=88c982a517e8e1e8394a470ee12efd7a8cfa0

In [79]:
import ir_datasets

# Load the dataset
dataset = ir_datasets.load("msmarco-passage/dev")

# Basic information about the dataset
print(f"Dataset name: {dataset.dataset_id()}")
print(f"Has queries: {dataset.has_queries()}")
print(f"Has docs: {dataset.has_docs()}")
print(f"Has qrels: {dataset.has_qrels()}")

# Count the queries
queries_count = sum(1 for _ in dataset.queries_iter())
print(f"Number of queries: {queries_count}")

Dataset name: msmarco-passage/dev
Has queries: True
Has docs: True
Has qrels: True


[INFO] Please confirm you agree to the MSMARCO data usage agreement found at <http://www.msmarco.org/dataset.aspx>
[INFO] If you have a local copy of https://msmarco.z22.web.core.windows.net/msmarcoranking/queries.tar.gz, you can symlink it here to avoid downloading it again: /Users/uvuv/.ir_datasets/downloads/c177b2795d5f2dcc524cf00fcd973be1
[INFO] [starting] https://msmarco.z22.web.core.windows.net/msmarcoranking/queries.tar.gz
[INFO] [finished] https://msmarco.z22.web.core.windows.net/msmarcoranking/queries.tar.gz: [00:03] [18.9MB] [6.06MB/s]


Number of queries: 101093


In [83]:
# Iterate through queries
print("\n=== First 5 queries ===")
for i, query in enumerate(dataset.queries_iter()):
    if i >= 5:  # Show only first 5
        break
    print(f"Query ID: {query.query_id}, Text: {query.text}")



=== First 5 queries ===
Query ID: 1048578, Text: cost of endless pools/swim spa
Query ID: 1048579, Text: what is pcnt
Query ID: 1048580, Text: what is pcb waste
Query ID: 1048581, Text: what is pbis?
Query ID: 1048582, Text: what is paysky


In [92]:
print("=== Примеры релевантных пар query-passage ===")
for i, qrel in enumerate(dataset.qrels_iter()):
    if i >= 10:  # Покажем первые 10
        break
    print(f"Query {qrel.query_id} -> Passage {qrel.doc_id}, Relevance: {qrel.relevance}")

=== Примеры релевантных пар query-passage ===
Query 1102432 -> Passage 2026790, Relevance: 1
Query 1102431 -> Passage 7066866, Relevance: 1
Query 1102431 -> Passage 7066867, Relevance: 1
Query 1090282 -> Passage 7066900, Relevance: 1
Query 39449 -> Passage 7066905, Relevance: 1
Query 76162 -> Passage 7066915, Relevance: 1
Query 195512 -> Passage 7066971, Relevance: 1
Query 1090280 -> Passage 7067004, Relevance: 1
Query 331318 -> Passage 5309290, Relevance: 1
Query 300674 -> Passage 7067032, Relevance: 1


In [ ]:
print("=== Примеры релевантных пар query-passage ===")
for i, qrel in enumerate(dataset.qrels_iter()):
    if i >= 10:  # Покажем первые 10
        break
    print(f"Query {qrel.query_id} -> Passage {qrel.doc_id}, Relevance: {qrel.relevance}")

In [ ]:
qrels_mapping = {}
for qrel in dataset.qrels_iter():
    qrels_mapping.setdefault(qrel.query_id, {})[qrel.doc_id] = qrel.relevance


In [107]:
dd = {
    "endless": [("8003157", 1), ("8003158", 2), ("8003159", 3)]
}

for query in dataset.queries_iter():
    query_id = query.query_id
    relevant_passages = qrels_mapping.get(query_id, {})
    
    if len(relevant_passages) > 0:

        with torch.no_grad():
            query_rep = model(d_kwargs=tokenizer(query.text, return_tensors="pt"))["d_rep"].squeeze()

        col = torch.nonzero(query_rep).squeeze().cpu().tolist()
        weights = query_rep[col].cpu().tolist()
        d = {k: v for k, v in zip(col, weights)}
        sorted_d = {k: v for k, v in sorted(d.items(), key=lambda item: item[1], reverse=True)}
        bow_rep = []
        for k, v in sorted_d.items():
            bow_rep.append((reverse_voc[k], v))
        
        g_d = {}
        for q, v in bow_rep:
            if q in dd:
                for doc_id, doc_relevance in dd[q]:
                    g_d[doc_id] = g_d.get(doc_id, 0) + doc_relevance * v

        break

        # relevant_indicies = torch.argsort(query_rep @ doc_rep.T, descending=True)
        # result_index = torch.argmax(torch.tensor(df['train']['passages'][index]['is_selected']))
        # rank = torch.where(relevant_indicies == result_index)
        # mrr10 += 1 / (rank[0] + 1)

        # print(relevant_passages)

g_d

/Users/uvuv/Informational_Search/spladev2/cloned/splade/splade/models/transformer_rep.py:89: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast() if self.fp16 else NullContextManager():
/Users/uvuv/Informational_Search/.venv/lib/python3.12/site-packages/torch/amp/autocast_mode.py:270: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(
/Users/uvuv/Informational_Search/spladev2/cloned/splade/splade/models/transformer_rep.py:32: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast() if self.fp16 else NullContextManager():


{'8003157': 2.2332911491394043,
 '8003158': 4.466582298278809,
 '8003159': 6.699873447418213}